# Neural net

In [46]:
#install.packages("keras")
library(keras)
#keras::install_keras(method='conda', envname = 'ctDNA', conda="home/anikago/miniconda3/bin/conda")

#install.packages("cvms")
#library(cvms)

library(dplyr)

#install.packages("tensorflow")
library(tensorflow)

In [30]:
sum_control_ATAC_bin_rm = read.csv("sum_control_ATAC_bin_rm.csv")

Function for the neural net

In [47]:
# Small neural network

fit_neural_net <- function(x_train, y_train, batch_size=128, epoch=100, lr=0.001, dropout=.1){

  # Define model for regression
  # We create 3 layers
  #   1: Has 128 neurons and is activated by relu (sets negative values to 0)
  #   2: Has 32 neurons and is activated by relu
  #   3: Has 1 neuron (the output value) and is activated by softplus
  #      The softplus activation function makes the output positive
  #      but I'm not sure whether it is better than e.g. relu or something else
  model <- keras_model_sequential() %>%
    layer_dropout(rate = dropout, input_shape = ncol(x_train)) %>%
    layer_dense(units = 128, activation = 'relu') %>%
    layer_dropout(rate = dropout) %>%
    layer_dense(units = 32, activation = 'relu') %>%
    layer_dropout(rate = dropout) %>%
    layer_dense(units = 1, activation = 'softplus')

  # Compile model
  model %>% compile(
    loss = loss_mean_squared_error,
    optimizer = optimizer_adam(learning_rate = lr),
    metrics = metric_mean_squared_error
  )

  # Train model
  model %>% fit(
    x_train, y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split = 0.2
  )

  model
}

In [48]:
# Fit the neural net
m <- fit_neural_net(
  x_train = sum_control_ATAC_bin_rm %>% dplyr::select(-ATAC_val),
  y_train = sum_control_ATAC_bin_rm %>% dplyr::select(ATAC_val),
  batch_size = 128,
  epoch = 100,
  lr = 0.001,
  dropout = .1
)

List of 19
 $ python              : chr "/home/anikago/miniconda3/envs/ctDNA/bin/python3"
 $ libpython           : chr "/home/anikago/miniconda3/envs/ctDNA/lib/libpython3.9.so"
 $ pythonhome          : chr "/home/anikago/miniconda3/envs/ctDNA:/home/anikago/miniconda3/envs/ctDNA"
 $ pythonpath          : chr "/home/anikago/miniconda3/envs/ctDNA/lib/python39.zip:/home/anikago/miniconda3/envs/ctDNA/lib/python3.9:/home/an"| __truncated__
 $ prefix              : chr "/home/anikago/miniconda3/envs/ctDNA"
 $ exec_prefix         : chr "/home/anikago/miniconda3/envs/ctDNA"
 $ base_exec_prefix    : chr "/home/anikago/miniconda3/envs/ctDNA"
 $ virtualenv          : chr ""
 $ virtualenv_activate : chr ""
 $ version_string      : chr "3.9.6 (default, Aug 18 2021, 19:38:01)  [GCC 7.5.0]"
 $ version             : chr "3.9"
 $ architecture        : chr "64bit"
 $ anaconda            : logi FALSE
 $ numpy               :List of 2
  ..$ path   : chr "/home/anikago/miniconda3/envs/ctDNA/lib/python3.9/si

ERROR: Error: Python module tensorflow.keras was not found.

Detected Python configuration:





In [ ]:



# Some random data
set.seed(1)
data_size <- 500
data <- tibble(
  x1 = sin(1:data_size / 10) + 0.1 * rnorm(data_size),
  x2 = cos(1:data_size / 10) + 0.1 * rnorm(data_size),
  x3 = rnorm(data_size),
  y = 0.4 * x1 + 0.7 * x2 + 2.4 * x3 + 0.03 * rnorm(data_size)
)

library(groupdata2)

parts <- groupdata2::partition(data, p = 0.2)
train <- parts[[2]]
test <- parts[[1]]


# Cross-validate a simple lm() gets us rmse=0.0316 (on my machine at least)
train_folded <- groupdata2::fold(train, k = 5)
cvms::cross_validate(train_folded,
                     "y ~ x1 + x2 + x3",
                     fold_cols = ".folds",
                     family = "gaussian")

# Fit the neural net
m <- fit_neural_net(
  x_train = train[, -4],
  y_train = train[, 4],
  batch_size = 128,
  epoch = 100,
  lr = 0.001,
  dropout = .1
)

# I think something like this is the code for predicting
predict(m, test[, -4])
